Chapter 2 Derivatives and Gradients
========

Example 2.1. Symbolic differentiation provides analytical derivatives.
----

In [1]:
import sympy as sp

In [2]:
x = sp.symbols('x')
f = x**2 + x/2 - sp.sin(x)/x
f_prime = sp.diff(f, x)

In [3]:
print("Function:")
sp.pprint(f)
print("\nDerivative:")
sp.pprint(f_prime)

Function:
 2   x   sin(x)
x  + ─ - ──────
     2     x   

Derivative:
      1   cos(x)   sin(x)
2⋅x + ─ - ────── + ──────
      2     x         2  
                     x   


2.3.1 Finite Difference Methods
----

In [4]:
import numpy as np

In [5]:
def diff_forward(f, x, h=1e-9):
    return (f(x + h) - f(x)) / h

def diff_central(f, x, h=1e-9):
    return (f(x + h/2) - f(x - h/2)) / h

def diff_backward(f, x, h=1e-9):
    return (f(x) - f(x - h)) / h

In [6]:
# Differentiate a sample function
f = np.sin
x = 1.0
h = 0.1  # Use a larger h to see differences

# Calculate derivatives 
print(f"Forward:  {diff_forward(f, x=x, h=h):.4f}")
print(f"Central:  {diff_central(f, x=x, h=h):.4f}")
print(f"Backward: {diff_backward(f, x=x, h=h):.4f}")
print(f"True:     {np.cos(x== 1.0):.4f}")

Forward:  0.4974
Central:  0.5401
Backward: 0.5814
True:     0.5405


2.3.2 Complex Methods
-----

In [7]:
def f(x):
    return np.sin(x**2)

x = np.pi / 2
h = 0.001
complex_input = x + h * 1j
v = f(complex_input)

In [9]:
f_x = v.real
print(f"f(x) ≈ {f_x:.4f}")

f_prime_x = v.imag / h
print(f"f'(x) ≈ {f_prime_x:.4f}")

f(x) ≈ 0.6243
f'(x) ≈ -2.4543


Example 2.5. An implementation of dual numbers that allows for automatic forward accumulation.
-----

In [8]:
import math

In [10]:
class Dual:
    """Dual number for automatic differentiation"""
    def __init__(self, v, d):
        self.v = v  # Real part
        self.d = d  # Dual part (derivative)
    
    def __add__(self, other):
        return Dual(self.v + other.v, self.d + other.d)
       
    def __radd__(self, other):
        return self.__add__(other)
    
    def __mul__(self, other):
        return Dual(self.v * other.v, self.v * other.d + other.v * self.d)
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    def __repr__(self):
        return f"Dual({self.v}, {self.d})"

def log_dual(a):
    """Logarithm for dual numbers"""
    return Dual(math.log(a.v), a.d / a.v)
        
def max_dual(a, b):
    """Maximum for dual numbers"""
    if isinstance(a, Dual) and isinstance(b, Dual):
        v = a.v if a.v > b.v else b.v
        if a.v > b.v:
            d = a.d
        elif a.v < b.v:
            d = b.d
        else:
            d = float('nan')
        return Dual(v, d)
    elif isinstance(a, Dual):
        v = a.v if a.v > b else b
        if a.v > b:
            d = a.d
        elif a.v < b:
            d = 0
        else:
            d = float('nan')
        return Dual(v, d)

In [11]:
# Example
a = Dual(3, 1)
b = Dual(2, 0)
print(log_dual(a * b + max_dual(a, 2)))  

Dual(2.1972245773362196, 0.3333333333333333)
